In [1]:
import struct
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pywt
import six

In [2]:
def get_file(path):
    global filelist
    filelisttemp = os.listdir(path)
    for filename in filelisttemp:
        if os.path.isdir(path + "/" + filename):
            get_file(path + "/" + filename)
        else:
            filelist.append(path + "/" + filename)

In [3]:
def save_file(writer):
    interval = 5
    dt = 0.0000001 * interval
    fs = 10000000 / interval
    start = int(250000 / interval)
    end = int(350000 / interval)
    fig_size = 20

    global filelist
    for path in filelist:
        fb = open(path,'rb')
        x = 0
        data1 = []
        data2 = []
        while True:
            data = fb.read(4)
            if not data:
                break
            if x % interval == 0:
                ch1, ch2 = struct.unpack('<HH', data)
                ch1 = (float(ch1) - 8192) / 8192 * 2.5
                ch2 = (float(ch2) - 8192) / 8192 * 2.5
                ch1 = float(ch1)
                ch2 = float(ch2)
                data1.append(ch1)
                data2.append(ch2)
            x = x + 1

        fb.close()

        data1 = data1[start:end]
        data2 = data2[start:end]

        wavelet = 'morl'
        c = pywt.central_frequency(wavelet)
        fa = np.arange(400000, 20000 - 1, -20000)
        scales = np.array(float(c)) * fs / np.array(fa)

        [cfs1,frequencies1] = pywt.cwt(data1,scales,wavelet,dt)
        [cfs2,frequencies2] = pywt.cwt(data2,scales,wavelet,dt)
        power1 = (abs(cfs1)) ** 2
        power2 = (abs(cfs2)) ** 2

        length_now = len(power2[0])
        power1 = np.reshape(power1,(len(power1),fig_size,int(length_now/fig_size)))
        power2 = np.reshape(power2,(len(power2),fig_size,int(length_now/fig_size)))
        power1 = np.log10(np.mean(power1,axis=2))
        power2 = np.log10(np.mean(power2,axis=2))

        mx = power1.max()
        mn = power1.min()
        power1 = (power1-mn) / (mx-mn) * 255.0
        power1 = power1.astype(np.uint8)
        power1 = power1.tobytes()

        mx = power2.max()
        mn = power2.min()
        power2 = (power2-mn) / (mx-mn) * 255.0
        power2 = power2.astype(np.uint8)
        power2 = power2.tobytes()

        example = tf.train.Example(features=tf.train.Features(feature={'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[power1]))}))
        writer.write(example.SerializeToString())
        example = tf.train.Example(features=tf.train.Features(feature={'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[power2]))}))
        writer.write(example.SerializeToString())

In [4]:
filelist = []

recordname = "/home/adoge/AE-location/input_data/train1"
get_file(recordname)
recordname = "/home/adoge/AE-location/input_data/train_knock"
get_file(recordname)
filelist = np.array(filelist)
np.random.shuffle(filelist)

writer = tf.python_io.TFRecordWriter("VAE_input.tfrecord")
save_file(writer)

writer.close()

In [5]:
filelist

array(['/home/adoge/AE-location/input_data/train1/110/19',
       '/home/adoge/AE-location/input_data/train_knock/60/8',
       '/home/adoge/AE-location/input_data/train1/60/8',
       '/home/adoge/AE-location/input_data/train1/80/26',
       '/home/adoge/AE-location/input_data/train1/115/25',
       '/home/adoge/AE-location/input_data/train1/120/11',
       '/home/adoge/AE-location/input_data/train1/85/23',
       '/home/adoge/AE-location/input_data/train1/75/10',
       '/home/adoge/AE-location/input_data/train1/90/29',
       '/home/adoge/AE-location/input_data/train1/135/23',
       '/home/adoge/AE-location/input_data/train1/125/25',
       '/home/adoge/AE-location/input_data/train_knock/90/20',
       '/home/adoge/AE-location/input_data/train1/125/6',
       '/home/adoge/AE-location/input_data/train1/85/22',
       '/home/adoge/AE-location/input_data/train1/135/4',
       '/home/adoge/AE-location/input_data/train1/65/23',
       '/home/adoge/AE-location/input_data/train_knock/90/1